In [1]:
import json
import copy

annotator_list = ['alex', 'elizabeth', 'rachel', 'marcus', 'vishnesh']
annotator_data_list = []
total_original_sentences = []
annotator_original_sentences_list = []
for annotator_name in annotator_list:
    directory = '' + annotator_name + '/'
    original_sentences = []
    with open(directory + "total.json") as f:
        data = json.load(f)
        # itereate data to get total original sentences
        for data_instance in data:
            total_original_sentences.append(data_instance['Original'])
            original_sentences.append(data_instance['Original'])
        annotator_original_sentences_list.append(original_sentences)
        annotator_data_list.append(data)
    print((annotator_name, len(data)))


('alex', 99)
('elizabeth', 98)
('rachel', 99)
('marcus', 99)
('vishnesh', 100)


In [2]:
# read "adding_new_one.json"
with open("adding_new_one.json") as f:
    the_new_one = json.load(f)[0]
    the_new_one_original = the_new_one['Original']

be_replaced_one_original = "He died on May 29, 1518 in Madrid, Spain and was buried in the church of San Benito d'Alcantara."

In [3]:
# remove the annotation of be_replaced_one_original from each annotator's annotations
for i, annotator_name in enumerate(annotator_list):
    annotator_data = annotator_data_list[i]
    annotator_original_sentences = annotator_original_sentences_list[i]
    for j, data_instance in enumerate(annotator_data):
        if data_instance['Original'] == be_replaced_one_original:
            annotator_data.pop(j)
            annotator_original_sentences.pop(j)
            break

In [4]:
total_original_sentences = list(set(total_original_sentences))
# remove be_replaced_one_original from total_original_sentences
total_original_sentences.remove(be_replaced_one_original)
# add the_new_one_original to total_original_sentences
total_original_sentences.append(the_new_one_original)

In [5]:
# save total original sentences to file
with open('total_100_original_sentences.json', 'w') as f:
    json.dump(total_original_sentences, f)

In [6]:
vishnesh_annotations = annotator_data_list[-1]

In [7]:
for i, annotator_name in enumerate(annotator_list):
    operations = ["Deletions", "Paraphrases", "Splittings"]
    annotations = annotator_data_list[i]
    missing_originals = list(set(total_original_sentences) - set(annotator_original_sentences_list[i]))
    for missing_original in missing_originals:
        if missing_original == the_new_one_original:
            new_one_copy = copy.deepcopy(the_new_one)
            new_one_copy['Worker'] = annotator_name
            for j, operation in enumerate(operations):
                    for k, edit in enumerate(new_one_copy[operation]):
                        # add -1 to the edit at front
                        edit_temp = [-1] + edit
                        new_one_copy[operation][k] = edit_temp
            annotator_data_list[i].append(new_one_copy)
        else:
            for instance in vishnesh_annotations:
                original_sentence = instance['Original']
                if original_sentence == missing_original:
                    # deep copy instance as instance_copy
                    instance_copy = copy.deepcopy(instance)
                    instance_copy['Worker'] = annotator_name
                    for operation in operations:
                        for j, edit in enumerate(instance_copy[operation]):
                            instance_copy[operation][j][0] = -1
                    annotator_data_list[i].append(instance_copy)
                    break


In [8]:
for i, annotator_name in enumerate(annotator_list):
    print((annotator_name, len(annotator_data_list[i])))

('alex', 100)
('elizabeth', 100)
('rachel', 100)
('marcus', 100)
('vishnesh', 100)


In [9]:
# assign ids to instances
for i, annotator_name in enumerate(annotator_list):
    for annotation in annotator_data_list[i]:
        original_sentence = annotation['Original']
        annotation['ID'] = total_original_sentences.index(original_sentence)

In [10]:
# add three new references into the data

# read three_new_systems_input.json
with open('../systems/three_new_systems_input.json') as f:
    three_new_systems_input = json.load(f)

for annotation in three_new_systems_input:
    operations = ["Deletions", "Paraphrases", "Splittings"]
    original_sentence = annotation['Original']
    for i, annotator_name in enumerate(annotator_list):
        annotator_annotations = annotator_data_list[i]
        for annotator_annotation in annotator_annotations:
            if annotator_annotation['Original'] == original_sentence:
                for j, operation in enumerate(operations):
                    for k, edit in enumerate(annotation[operation]):
                        # add -1 to the edit at front
                        edit_temp = [-1] + edit
                        # add to annotator_annotation
                        annotator_annotation[operation].append(edit_temp)

In [11]:
# split each annotator's data into 3 batches, id < 33, 33 <= id < 66, 66 <= id < 100
for i, annotator_name in enumerate(annotator_list):
    annotations = annotator_data_list[i]
    # sort annotations by id
    annotations.sort(key=lambda x: x['ID'])
    annotations_batch_1 = []
    annotations_batch_2 = []
    annotations_batch_3 = []
    for annotation in annotations:
        if annotation['ID'] < 33:
            annotations_batch_1.append(annotation)
        elif annotation['ID'] < 66:
            annotations_batch_2.append(annotation)
        else:
            annotations_batch_3.append(annotation)
    print(len(annotations_batch_1), len(annotations_batch_2), len(annotations_batch_3))
    with open(annotator_name + '/new_batches/batch_1.json', 'w') as f:
        json.dump(annotations_batch_1, f)
    with open(annotator_name + '/new_batches/batch_2.json', 'w') as f:
        json.dump(annotations_batch_2, f)
    with open(annotator_name + '/new_batches/batch_3.json', 'w') as f:
        json.dump(annotations_batch_3, f)

33 33 34
33 33 34
33 33 34
33 33 34
33 33 34


In [16]:
#  read batch 3 from elizabeth
with open('vishnesh/new_batches/batch_3.json') as f:
    elizabeth_batch_3 = json.load(f)

In [17]:
for j, annotation in enumerate(elizabeth_batch_3):
    operations = ["Deletions", "Paraphrases", "Splittings"]
    new_system_num = 0
    for operation in operations:
        for i, edit in enumerate(annotation[operation]):
            system = edit[2]
            if "new_systems" in system:
                new_system_num += 1
    if new_system_num != 3:
        print(annotation['Original'], annotation["ID"], j, new_system_num)